# Local Search
## Solving N-queen problem with min-conflict algorithm

* First generate a complete assignment for all variables (this set of assignments may conflict)

* Repeat the following steps until there are no conflicts:

    * Randomly Select a variable that causes conflicts
    
    * Reassign the value of this variable to another value that with the least constraint onflicts with other variables

In [5]:
import numpy as np
import time
import random


def my_range(start, end):
    if start <= end:
        return range(start, end + 1)
    else:
        return range(start, end - 1, -1)


class Problem:
    char_mapping = ('·', 'Q')

    def __init__(self, n=4):
        self.n = n

    def is_valid(self, state):
        """
        check the state satisfy condition or not.
        :param state: list of points (in (row id, col id) tuple form)
        :return: bool value of valid or not
        """
        board = self.get_board(state)
        res = True
        for point in state:
            i, j = point
            condition1 = board[:, j].sum() <= 1
            condition2 = board[i, :].sum() <= 1
            condition3 = self.pos_slant_condition(board, point)
            condition4 = self.neg_slant_condition(board, point)
            res = res and condition1 and condition2 and condition3 and condition4
            if not res:
                break
        return res

    def is_satisfy(self, state):
        return self.is_valid(state) and len(state) == self.n

    def next_action(self, point):
        i, j = point
        if 0 <= i < self.n and 0 <= j < self.n and i * self.n + j < self.n ** 2 - 1:
            j += 1
            if j == self.n:
                j = 0
                i += 1
            return i, j
        else:
            return None

    def pos_slant_condition(self, board, point):
        i, j = point
        tmp = min(self.n - i - 1, j)
        start = (i + tmp, j - tmp)
        tmp = min(i, self.n - j - 1)
        end = (i - tmp,  j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def neg_slant_condition(self, board, point):
        i, j = point
        tmp = min(i, j)
        start = (i - tmp, j - tmp)
        tmp = min(self.n - i - 1, self.n - j - 1)
        end = (i + tmp, j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def get_board(self, state):
        board = np.zeros([self.n, self.n], dtype=int)
        for point in state:
            board[point] = 1
        return board

    def print_state(self, state):
        board = self.get_board(state)
        print('_' * (2 * self.n + 1))
        for row in board:
            for item in row:
                print(f'|{Problem.char_mapping[item]}', end='')
            print('|')
        print('-' * (2 * self.n + 1))

In [6]:
def attack_cnt(board, pos, n):
    def my_range(start, end):
        if start <= end:
            return range(start, end + 1)
        else:
            return range(start, end - 1, -1)

    attack_board = np.zeros((n, n), dtype=bool)
    attack_board[:, pos[1]] = attack_board[pos[0], :] = True
    i, j = pos
    tmp = min(n - i - 1, j)
    start = (i + tmp, j - tmp)
    tmp = min(i, n - j - 1)
    end = (i - tmp,  j + tmp)
    rows = my_range(start[0], end[0])
    cols = my_range(start[1], end[1])
    attack_board[rows, cols] = True

    tmp = min(i, j)
    start = (i - tmp, j - tmp)
    tmp = min(n - i - 1, n - j - 1)
    end = (i + tmp, j + tmp)
    rows = my_range(start[0], end[0])
    cols = my_range(start[1], end[1])
    attack_board[rows, cols] = True
    
    return (board & attack_board).sum() - 1


In [7]:
def min_conflict(problem):
    n = problem.n
    bet = [i for i in range(n)]
    queens = bet[:]
    random.shuffle(queens)
    # queens = random.choices(bet, k=n)
    actions = list(zip(bet, queens))  # row, col
    while not problem.is_satisfy(actions):
        board = np.zeros((n, n), dtype=bool)
        actions_np = np.array(actions)
        board[actions_np[:, 0], actions_np[:, 1]] = True

        
        cnt = [(i, attack_cnt(board, actions[i], n)) for i in range(len(actions))]
        cnt = sorted(cnt, key=lambda x: x[1], reverse=True)
        cnt = cnt[:(np.array(cnt)[:, 1] > 0).sum()]
        a = random.choice(cnt)

        # 计算那一行的所有位置的冲突数
        board[a] = False # 拿走这个棋子
        row_attack_cnt = [(i, attack_cnt(board, (a[0], i), n)) for i in range(n)]
        random.shuffle(row_attack_cnt)
        min_attack_pos = min(row_attack_cnt, key=lambda x: x[1])

        actions[a[0]] = (a[0], min_attack_pos[0])
        
        yield actions

In [10]:
# test block
def test(n):
    render = (n == 15)
    p = Problem(n)
    if render:
        import pygame

        w, h = 90 * n + 10, 90 * n + 10
        screen = pygame.display.set_mode((w, h))
        screen.fill('white')
        action_generator = min_conflict(p)
        clk = pygame.time.Clock()
        queen_img = pygame.image.load('./queen_s.png')
        queen_img = pygame.transform.scale(queen_img, (80, 80))
        while True:
            for event in pygame.event.get():
                if event == pygame.QUIT:
                    exit()
            try:
                actions = next(action_generator)
                screen.fill('white')
                for i in range(n + 1):
                    pygame.draw.rect(screen, 'black', (i * 90, 0, 10, h))
                    pygame.draw.rect(screen, 'black', (0, i * 90, w, 10))
                for action in actions:
                    i, j = action
                    screen.blit(queen_img, (10 + 90 * j, 10 + 90 * i))
                pygame.display.flip()
            except StopIteration:
                pass
            clk.tick(5)
        pass
    else:
        start_time = time.time()
        for actions in min_conflict(p):
            pass
        print(time.time() - start_time)
        p.print_state(actions)

import cProfile
cProfile.run("test(100)")

1.7071614265441895
_________________________________________________________________________________________________________________________________________________________________________________________________________
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|